## eva_vs_uniform sampling 

In [1]:
import os
os.getcwd()

'/home/jbang36/eva_jaeho/eva_storage/evaluation/evaluate_rep'

In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

import os
import sys
sys.argv=['']
sys.path.append('../../../')


import numpy as np
#import utils.helpers as helpers
import utils as helpers
from loaders.uadetrac_loader import UADetracLoader
from eva_storage.preprocessingModule import PreprocessingModule
from eva_storage.UNet import UNet
from eva_storage.clusterModule import ClusterModule
from filters.minimum_filter import FilterMinimum
from eva_storage.evaluation.evaluate_compression import *
from eva_storage.models.Autoencoder import Autoencoder


Populating the interactive namespace from numpy and matplotlib


In [3]:
import torch
#torch.cuda.set_device(0) # we want to use both gpus

In [4]:
loader = UADetracLoader()
images = loader.load_cached_images()
video_start_indices = loader.get_video_start_indices()
labels = loader.load_cached_labels()

## done loading the dataset

In [ ]:
## assume we already have preprocessed the images + trained the network
## if not, look at eva_vs_nop__ml.ipynb to find out how to do this

In [5]:
# load the network
ae = Autoencoder().cuda()
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(ae.parameters(), lr=1e-3)


In [6]:
#create a dataset object 
images_normalized = images.astype(np.float32)
images_normalized /= 255.0
train_data = torch.from_numpy(images_normalized)
train_data = train_data.permute(0, 3, 1, 2)


In [7]:
batch_size = 128
image_dataset = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)

In [8]:
losses = []
EPOCHS = 40
for epoch in range(EPOCHS):
    for batch_idx, data in enumerate(image_dataset):   
        
        
        data = data.cuda()
        #print(data.size())
        compressed, final = ae(data)

        optimizer.zero_grad()
        loss = loss_func(final, data)
        loss.backward()
        optimizer.step()
        
    
    print('Train Epoch: {}/{} \tLoss: {:.6f}'.format(
        epoch+1,
        EPOCHS, 
        loss.cuda().data.item()), 
        end='\n')


Train Epoch: 1/40 	Loss: 0.014788
Train Epoch: 2/40 	Loss: 0.007426
Train Epoch: 3/40 	Loss: 0.006745
Train Epoch: 4/40 	Loss: 0.006359
Train Epoch: 5/40 	Loss: 0.006019
Train Epoch: 6/40 	Loss: 0.005699
Train Epoch: 7/40 	Loss: 0.005427
Train Epoch: 8/40 	Loss: 0.005208
Train Epoch: 9/40 	Loss: 0.005021
Train Epoch: 10/40 	Loss: 0.004869
Train Epoch: 11/40 	Loss: 0.004726
Train Epoch: 12/40 	Loss: 0.004616
Train Epoch: 13/40 	Loss: 0.004516
Train Epoch: 14/40 	Loss: 0.004410
Train Epoch: 15/40 	Loss: 0.004317
Train Epoch: 16/40 	Loss: 0.004236
Train Epoch: 17/40 	Loss: 0.004168
Train Epoch: 18/40 	Loss: 0.004089
Train Epoch: 19/40 	Loss: 0.004027
Train Epoch: 20/40 	Loss: 0.003973
Train Epoch: 21/40 	Loss: 0.003927
Train Epoch: 22/40 	Loss: 0.003876
Train Epoch: 23/40 	Loss: 0.003829
Train Epoch: 24/40 	Loss: 0.003788
Train Epoch: 25/40 	Loss: 0.003750
Train Epoch: 26/40 	Loss: 0.003709
Train Epoch: 27/40 	Loss: 0.004204
Train Epoch: 28/40 	Loss: 0.003655
Train Epoch: 29/40 	Loss: 0.0

In [10]:
### saving the trained network
save_directory = '/home/jbang36/eva_jaeho/models/ae.pth'
print("Saving the trained model as....", save_directory)

torch.save(ae.state_dict(), save_directory)



Saving the trained model as.... /home/jbang36/eva_jaeho/models/ae.pth


In [ ]:
### loading the trained network
save_directory = '/home/jbang36/eva_jaeho/models/ae.pth'  
ae = Autoencoder()
ae.load_state_dict(torch.load(save_directory))
ae = ae.cuda()
print("Model successfully loaded!")


In [11]:
compressed_data = np.ndarray(shape=(train_data.shape[0], 9, 75, 75)) 
for i, images in enumerate(image_dataset):
    images = images.cuda()
    compressed, final = ae(images)
    compressed_cpu = compressed.detach().cpu().numpy()
    compressed_cpu *= 255
    compressed_cpu = compressed_cpu.astype(np.uint8)
    compressed_data[i * batch_size:(i + 1) * batch_size] = compressed_cpu

In [10]:
compressed_data.shape

(25435, 9, 75, 75)

In [12]:
## too lazy to change variables....
unet_compressed_images = compressed_data

#need to flatten the data
unet_compressed_images = unet_compressed_images.reshape(train_data.shape[0], 9*75*75)

In [13]:
unet_compressed_images.shape

(25435, 50625)

In [21]:
a = np.array([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]])
b = a[:,::2]
print(a)
print("-------------")
print(b)

[[1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]]
-------------
[[1 3 5]
 [1 3 5]
 [1 3 5]
 [1 3 5]
 [1 3 5]]


In [23]:
## we want to downsample this because 50625 features is way too big
## how about something like 9 * 5 * 5??
unet_compressed_images = compressed_data[:,::25,::25]
unet_compressed_images = unet_compressed_images.reshape(train_data.shape[0], -1)
print(unet_compressed_images.shape)


(25435, 225)


In [24]:
# Create clusters
cm = ClusterModule()
image_cluster_labels = cm.run(unet_compressed_images)


Time to fit  25435 :  90.66402435302734


In [25]:
# Generate binary labels
## within labels['vehicle'] there are ['car', 'others', 'van', 'bus']

car_labels = helpers.generateBinaryLabels(labels['vehicle'])
other_labels = helpers.generateBinaryLabels(labels['vehicle'], label_of_interest = 'others')
van_labels = helpers.generateBinaryLabels(labels['vehicle'], 'van')
bus_labels = helpers.generateBinaryLabels(labels['vehicle'], 'bus')

In [52]:
## divide the training and validation dataset for all frames
division_point = int(unet_compressed_images.shape[0] * 0.8)
train_set = {}
test_set = {}
    
# this set contains [image, labels, cluster_labels]
train_set['van'] = [train_data[:division_point], van_labels[:division_point], image_cluster_labels[:division_point]]
train_set['bus'] = [train_data[:division_point], bus_labels[:division_point], image_cluster_labels[:division_point]]
test_set['van'] = [train_data[division_point:], van_labels[division_point:], image_cluster_labels[division_point:]]
test_set['bus'] = [train_data[division_point:], bus_labels[division_point:], image_cluster_labels[division_point:]]


In [53]:
print(division_point)
print(train_data.shape)
print(unet_compressed_images.shape)

20348
torch.Size([25435, 3, 300, 300])
(25435, 225)


In [34]:
train_data.shape

torch.Size([25435, 3, 300, 300])

In [54]:
## divide the training and validation dataset for rep frames
#division_point = int(rep_images.shape[0] * 0.8)

rep_train_set = {}
rep_test_set = {}
for key in ['van', 'bus']:
    rep_train_set[key] = get_rep_frames(train_set[key][0], train_set[key][1], train_set[key][2])
    rep_test_set[key] = get_rep_frames(test_set[key][0], test_set[key][1], test_set[key][2])
    

In [55]:
# Test on filters
fm_everyframe_set = {}
fm_repframe_set = {}
for key in ['van', 'bus']:
    print("training", key, ".....")
    fm_everyframe_set[key] = FilterMinimum()
    fm_everyframe_set[key].train(train_set[key][0], train_set[key][1])
    
    fm_repframe_set[key] = FilterMinimum()
    fm_repframe_set[key].train(rep_train_set[key][0], rep_train_set[key][1])
    

training van .....
torch.Size([16278, 270000])
(16278,)
(1016, 270000)
(1016,)
training bus .....
torch.Size([16278, 270000])
(16278,)
(1016, 270000)
(1016,)


In [56]:
# Evaluate the filters
# add y_hat as the third element in the test_set
# test set [images, labels, cluster labels, predicted labels]

for key in ['van', 'bus']:
    test_set[key].append(fm_everyframe_set[key].predict(test_set[key][0], post_model_name= 'rf'))
    rep_test_set[key] = list(rep_test_set[key])
    rep_test_set[key].append(fm_repframe_set[key].predict(rep_test_set[key][0], post_model_name = 'rf'))
    



In [57]:
## compute all frame score

for key in ['van', 'bus']:
    tmp = np.array(test_set[key][1] - test_set[key][3])
    all_frame_score = len(tmp[tmp == 0]) / float(len(tmp))
    print(key, "score is", all_frame_score * 100, "%")


van score is 43.876548063691764 %
bus score is 59.68154118340869 %


In [58]:
## compute rep frame score
# this requires me to assess -> spread the labels -> compare against original results

def extend_labels(y_hat_rep, X_test_clusters):
    arr = np.zeros(shape = (len(X_test_clusters)))
    
    for i in range(len(X_test_clusters)):
        arr[i] = y_hat_rep[X_test_clusters[i]]
    return arr


In [59]:
for key in ['van', 'bus']:
    y_hat = extend_labels(rep_test_set[key][2], test_set[key][2])
    tmp = np.array(y_hat - test_set[key][1])
    rep_frame_score = len(tmp[tmp == 0]) / float(len(tmp))
    print(key, "score is", rep_frame_score * 100, "%")
    

van score is 44.44662866129349 %
bus score is 48.67308826420287 %


## uniform sampling

In [61]:
### need logic to perform comparison vs uniform sampling

fps = 20
uniform_indexes_train = [i for i in range(0, division_point, fps)]
uniform_indexes_test = [(i - division_point) for i in range(division_point, end_frame, fps)] 


In [63]:
print(unet_compressed_images.shape)

(25435, 225)


In [64]:
print(end_frame)

25435


In [66]:
print(len(uniform_indexes_train))
print(len(uniform_indexes_test))

1018
255


In [67]:
## steps
## 1. get the corresponding labels, train the models, evaluate the model, and extend the labels

## this set contains [image, labels, cluster_labels]
## divide the training and validation dataset for rep frames
## division_point = int(rep_images.shape[0] * 0.8)

sample_train_set = {}
sample_test_set = {}
for key in ['van', 'bus']:
    sample_train_set[key] = (train_set[key][0][uniform_indexes_train], train_set[key][1][uniform_indexes_train]) ##rep_images, rep_labels get_rep_frames(train_set[key][0], train_set[key][1], train_set[key][2])
    sample_test_set[key] = (test_set[key][0][uniform_indexes_test], test_set[key][1][uniform_indexes_test])
    


In [68]:
print(sample_train_set['van'][0].shape)
print(sample_test_set['van'][0].shape)

torch.Size([1018, 3, 300, 300])
torch.Size([255, 3, 300, 300])


In [69]:
# Test on filters
fm_uniframe_set = {}
for key in ['van', 'bus']:
    print("training", key, ".....")
    fm_uniframe_set[key] = FilterMinimum()
    fm_uniframe_set[key].train(sample_train_set[key][0], sample_train_set[key][1])
    
    

training van .....
torch.Size([814, 270000])
(814,)
training bus .....
torch.Size([814, 270000])
(814,)


In [70]:
# Evaluate the filters
# add y_hat as the third element in the test_set
# test set [images, labels, cluster labels, predicted labels]

for key in ['van', 'bus']:
    #test_set[key].append(fm_everyframe_set[key].predict(test_set[key][0], post_model_name= 'rf'))
    sample_test_set[key] = list(sample_test_set[key])
    sample_test_set[key].append(fm_uniframe_set[key].predict(sample_test_set[key][0], post_model_name = 'rf'))
    


In [71]:
## compute all frame score

for key in ['van', 'bus']:
    tmp = np.array(test_set[key][1] - test_set[key][3])
    all_frame_score = len(tmp[tmp == 0]) / float(len(tmp))
    print(key, "score is", all_frame_score * 100, "%")


van score is 43.876548063691764 %
bus score is 59.68154118340869 %


In [72]:
### need to extend the labels
## TODO: modify this function for sampling
## for sampling, we will assume each 'cluster' is 
## from the starting frame to the frame right before the end


def extend_labels_sampling(y_hat_samp, images):
    end_frame = len(images)
    fps = 20
    arr = np.zeros(shape = (len(images)))
    print(arr.shape)
    print(len(y_hat_samp))
    
    j = -1
    for i in range(len(images)):
        if i % fps == 0:
            j += 1
        arr[i] = y_hat_samp[j]
    
    return arr


In [74]:
for key in ['van', 'bus']:
    y_hat = extend_labels_sampling(sample_test_set[key][2], unet_compressed_images[division_point:])
    tmp = np.array(y_hat - test_set[key][1])
    rep_frame_score = len(tmp[tmp == 0]) / float(len(tmp))
    print(key, "score is", rep_frame_score * 100, "%")

(5087,)
255
van score is 60.19264792608611 %
(5087,)
255
bus score is 46.03892274425005 %


## What is the distribution of the data??

In [75]:
""" 
train_set['van']
train_set['bus']


"""
for key in ['van', 'bus']:
    label = test_set[key][1]
    positive = sum(label) * 1.0 / len(label)
    print(key, "positive percentage is", positive * 100, "%")


van positive percentage is 57.89266758403774 %
bus positive percentage is 65.5396107725575 %
